In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the InceptionV3 model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
inception_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=cnn_input)

# Freeze the layers
for layer in inception_model.layers:
    layer.trainable = False

# Get the output of the InceptionV3 model
inception_output = inception_model.output

# Flatten the output
flatten_layer = Flatten()(inception_output)

# Define the DenseNet121 model for feature extraction
densenet_input = Input(shape=(img_width, img_height, 3))
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=densenet_input)

# Freeze the layers
for layer in densenet_model.layers:
    layer.trainable = False

# Get the output of the DenseNet121 model
densenet_output = densenet_model.output

# Reshape the densenet_output to match the shape of flatten_layer
reshaped_densenet_output = tf.keras.layers.GlobalAveragePooling2D()(densenet_output)

# Concatenate the outputs of InceptionV3 and DenseNet121
concatenated_output = Concatenate()([flatten_layer, reshaped_densenet_output])

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, concatenated_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(tf.expand_dims(concatenated_output, axis=1))


# Combine the InceptionV3, DenseNet121, and LSTM models
combined_model = Model(inputs=[cnn_input, densenet_input], outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2)

# Save the trained model
combined_model.save('lung_detection_inceptionv3_densenet121.h5')

Found 16050 images belonging to 6 classes.
Epoch 1/2


ValueError: in user code:

    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Hi\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 219, in assert_input_compatibility
        raise ValueError(

    ValueError: La

In [ ]:
# Set the path to your test dataset
test_data_dir = r"D:\Downloads\Dataset\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(
    test_generator,
    steps=len(test_generator))

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


In [13]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\test-20230326T155708Z-001\test\COVID\COVID-19 (8).jpeg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 423ms/step
Predicted class: COVID


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Define transforms for the images
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset
train_data = datasets.ImageFolder('D:/Downloads/Dataset/train', transform=data_transform)
val_data = datasets.ImageFolder('D:/Downloads/Dataset/val', transform=data_transform)
test_data = datasets.ImageFolder('D:/Downloads/Dataset/test', transform=data_transform)

# Define the data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the models
densenet121 = models.densenet121(pretrained=True)
inceptionv3 = models.inception_v3(pretrained=True)

# Freeze the parameters
for param in densenet121.parameters():
    param.requires_grad = False
for param in inceptionv3.parameters():
    param.requires_grad = False

# Modify the classifiers
num_ftrs_densenet121 = densenet121.classifier.in_features
densenet121.classifier = nn.Sequential(
    nn.Linear(num_ftrs_densenet121, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, len(train_data.classes))
)

num_ftrs_inceptionv3 = inceptionv3.fc.in_features
inceptionv3.fc = nn.Sequential(
    nn.Linear(num_ftrs_inceptionv3, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, len(train_data.classes))
)

# Combine the models in an ensemble
class Ensemble(nn.Module):
    def __init__(self, model1, model2):
        super(Ensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(len(train_data.classes)*2, len(train_data.classes))

    def forward(self, x):
        x1 = self.model1(x)
        x2 = self.model2(x)
        x = torch.cat((x1, x2), dim=1)
        x = self.fc(x)
        return x

ensemble_model = Ensemble(densenet121, inceptionv3)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model.to(device)

for epoch in range(num_epochs):
    ensemble_model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    ensemble_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = ensemble_model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss/len(val_loader):.4f}, Accuracy: {(correct/total)*100:.2f}%')

# Testing loop
ensemble_model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {(correct/total)*100:.2f}%')


RuntimeError: Calculated padded input size per channel: (3 x 3). Kernel size: (5 x 5). Kernel size can't be greater than actual input size

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Define ensemble model using DenseNet121 and InceptionV3
def ensemble_model(input_shape=(224, 224, 3), num_classes=6):
    # Load pre-trained DenseNet121 model
    densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in densenet_model.layers:
        layer.trainable = False

    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in inception_model.layers:
        layer.trainable = False

    # Define input layer
    input_layer = layers.Input(shape=input_shape)

    # Extract features using DenseNet121
    densenet_features = densenet_model(input_layer)
    densenet_features = layers.GlobalAveragePooling2D()(densenet_features)

    # Extract features using InceptionV3
    inception_features = inception_model(input_layer)
    inception_features = layers.GlobalAveragePooling2D()(inception_features)

    # Concatenate features
    merged_features = layers.Concatenate()([densenet_features, inception_features])

    # Add a fully connected layer
    merged_features = layers.Dense(512, activation='relu')(merged_features)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(merged_features)

    # Create model
    model = Model(inputs=input_layer, outputs=output)

    return model

# Define data directories
train_dir = r'D:\Downloads\Dataset\train'
val_dir = r'D:\Downloads\Dataset\val'
test_dir = r'D:\Downloads\Dataset\test'
image_size = (224, 224)
batch_size = 32

# Create data generators for training, validation, and testing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

# Load ensemble model
model = ensemble_model(input_shape=image_size + (3,), num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.
Epoch 1/10
200/200 [==============================] - 1421s 7s/step - loss: 1.1096 - accuracy: 0.6116 - val_loss: 0.8752 - val_accuracy: 0.6674
Epoch 2/10
200/200 [==============================] - 1329s 7s/step - loss: 0.7986 - accuracy: 0.7005 - val_loss: 0.7360 - val_accuracy: 0.7381
Epoch 3/10
200/200 [==============================] - 1348s 7s/step - loss: 0.7471 - accuracy: 0.7252 - val_loss: 0.6498 - val_accuracy: 0.7610
Epoch 4/10
200/200 [==============================] - 1319s 7s/step - loss: 0.6872 - accuracy: 0.7484 - val_loss: 0.6919 - val_accuracy: 0.7398
Epoch 5/10
200/200 [==============================] - 21040s 106s/step - loss: 0.6582 - accuracy: 0.7552 - val_loss: 0.5954 - val_accuracy: 0.7788
Epoch 6/10
200/200 [==============================] - 1341s 7s/step - loss: 0.6446 - accuracy: 0.7689 - val_loss: 0.5837 - val_accuracy: 0.7863
Epoch 7/10
200/200 [============================

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define custom DenseNet121 model
def custom_densenet121(input_shape=(224, 224, 3), num_classes=6):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    # Convolutional layers
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    # Dense blocks
    x = dense_block(x, 64, 6)
    x = transition_layer(x, 128)
    x = dense_block(x, 128, 12)
    x = transition_layer(x, 256)
    x = dense_block(x, 256, 24)
    x = transition_layer(x, 512)
    x = dense_block(x, 512, 16)
    # Global average pooling and classifier
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, x)
    return model

# Helper function for dense blocks
def dense_block(x, filters, blocks):
    for _ in range(blocks):
        x = conv_block(x, filters)
    return x

# Helper function for convolutional blocks within dense blocks
def conv_block(x, filters):
    x1 = layers.BatchNormalization()(x)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(filters, 3, padding='same')(x1)
    x = layers.Concatenate()([x, x1])
    return x

# Helper function for transition layers
def transition_layer(x, filters):
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 1)(x)
    x = layers.AveragePooling2D(2)(x)
    return x

# Define custom InceptionV3 model
def custom_inceptionv3(input_shape=(224, 224, 3), num_classes=6):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    # Stem
    x = inception_stem(x)
    # Inception blocks
    for _ in range(3):
        x = inception_block_a(x)
    x = reduction_a(x)
    for _ in range(4):
        x = inception_block_b(x)
    x = reduction_b(x)
    for _ in range(2):
        x = inception_block_c(x)
    # Top
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, x)
    return model

# Helper functions for InceptionV3 blocks
def inception_stem(x):
    x = layers.Conv2D(32, 3, strides=2, padding='valid', activation='relu')(x)
    x = layers.Conv2D(32, 3, strides=1, padding='valid', activation='relu')(x)
    x = layers.Conv2D(64, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(3, strides=2)(x)
    x = layers.Conv2D(80, 1, strides=1, padding='valid', activation='relu')(x)
    x = layers.Conv2D(192, 3, strides=1, padding='valid', activation='relu')(x)
    x = layers.MaxPooling2D(3, strides=2)(x)
    return x

def inception_block_a(x):
    branch1x1 = layers.Conv2D(64, 1, activation='relu')(x)

    branch5x5 = layers.Conv2D(48, 1, activation='relu')(x)
    branch5x5 = layers.Conv2D(64, 5, padding='same', activation='relu')(branch5x5)

    branch3x3dbl = layers.Conv2D(64, 1, activation='relu')(x)
    branch3x3dbl = layers.Conv2D(96, 3, padding='same', activation='relu')(branch3x3dbl)
    branch3x3dbl = layers.Conv2D(96, 3, padding='same', activation='relu')(branch3x3dbl)

    branch_pool = layers.AveragePooling2D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv2D(32, 1, activation='relu')(branch_pool)

    x = layers.Concatenate()([branch1x1, branch5x5, branch3x3dbl, branch_pool])
    return x

def reduction_a(x):
    branch3x3 = layers.Conv2D(384, 3, strides=2, padding='valid', activation='relu')(x)

    branch3x3dbl = layers.Conv2D(64, 1, activation='relu')(x)
    branch3x3dbl = layers.Conv2D(96, 3, padding='same', activation='relu')(branch3x3dbl)
    branch3x3dbl = layers.Conv2D(96, 3, strides=2, padding='valid', activation='relu')(branch3x3dbl)

    branch_pool = layers.MaxPooling2D(3, strides=2, padding='valid')(x)

    x = layers.Concatenate()([branch3x3, branch3x3dbl, branch_pool])
    return x

def inception_block_b(x):
    branch1x1 = layers.Conv2D(192, 1, activation='relu')(x)

    branch7x7 = layers.Conv2D(128, 1, activation='relu')(x)
    branch7x7 = layers.Conv2D(128, (1, 7), padding='same', activation='relu')(branch7x7)
    branch7x7 = layers.Conv2D(192, (7, 1), padding='same', activation='relu')(branch7x7)

    branch7x7dbl = layers.Conv2D(128, 1, activation='relu')(x)
    branch7x7dbl = layers.Conv2D(128, (7, 1), padding='same', activation='relu')(branch7x7dbl)
    branch7x7dbl = layers.Conv2D(128, (1, 7), padding='same', activation='relu')(branch7x7dbl)
    branch7x7dbl = layers.Conv2D(128, (7, 1), padding='same', activation='relu')(branch7x7dbl)
    branch7x7dbl = layers.Conv2D(192, (1, 7), padding='same', activation='relu')(branch7x7dbl)

    branch_pool = layers.AveragePooling2D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv2D(192, 1, activation='relu')(branch_pool)

    x = layers.Concatenate()([branch1x1, branch7x7, branch7x7dbl, branch_pool])
    return x

def reduction_b(x):
    branch3x3 = layers.Conv2D(192, 1, activation='relu')(x)
    branch3x3 = layers.Conv2D(320, 3, strides=2, padding='valid', activation='relu')(branch3x3)

    branch7x7x3 = layers.Conv2D(192, 1, activation='relu')(x)
    branch7x7x3 = layers.Conv2D(192, (1, 7), padding='same', activation='relu')(branch7x7x3)
    branch7x7x3 = layers.Conv2D(192, (7, 1), padding='same', activation='relu')(branch7x7x3)
    branch7x7x3 = layers.Conv2D(192, 3, strides=2, padding='valid', activation='relu')(branch7x7x3)

    branch_pool = layers.MaxPooling2D(3, strides=2, padding='valid')(x)

    x = layers.Concatenate()([branch3x3, branch7x7x3, branch_pool])
    return x

def inception_block_c(x):
    branch1x1 = layers.Conv2D(320, 1, activation='relu')(x)

    branch3x3 = layers.Conv2D(384, 1, activation='relu')(x)
    branch3x3_1 = layers.Conv2D(384, (1, 3), padding='same', activation='relu')(branch3x3)
    branch3x3_2 = layers.Conv2D(384, (3, 1), padding='same', activation='relu')(branch3x3)

    branch3x3dbl = layers.Conv2D(448, 1, activation='relu')(x)
    branch3x3dbl = layers.Conv2D(384, 3, padding='same', activation='relu')(branch3x3dbl)
    branch3x3dbl_1 = layers.Conv2D(384, (1, 3), padding='same', activation='relu')(branch3x3dbl)
    branch3x3dbl_2 = layers.Conv2D(384, (3, 1), padding='same', activation='relu')(branch3x3dbl)

    branch_pool = layers.AveragePooling2D(3, strides=1, padding='same')(x)
    branch_pool = layers.Conv2D(192, 1, activation='relu')(branch_pool)

    x = layers.Concatenate()([branch1x1, branch3x3_1, branch3x3_2, branch3x3dbl_1, branch3x3dbl_2, branch_pool])
    return x

# Define ensemble model
def ensemble_model(models):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    outputs = [model(inputs) for model in models]
    ensemble_output = layers.Average()(outputs)
    ensemble_model = Model(inputs, ensemble_output)
    return ensemble_model

# Build custom DenseNet121 and InceptionV3 models
densenet_model = custom_densenet121()
inception_model = custom_inceptionv3()

# Create an ensemble of the two models
ensemble_model = ensemble_model([densenet_model, inception_model])

# Print summary of the ensemble model
ensemble_model.summary()

# Compile the ensemble model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble model
history = ensemble_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the ensemble model on the test data
test_loss, test_accuracy = ensemble_model.evaluate(test_generator, steps=len(test_generator))
print("Test Accuracy:", test_accuracy)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_1 (Functional)        (None, 6)                    5145912   ['input_6[0][0]']             
                                                          38                                      
                                                                                                  
 model_2 (Functional)        (None, 6)                    1999242   ['input_6[0][0]']             
                                                          2                                       
                                                                                            

KeyboardInterrupt: 

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the dataset directory
data_dir = 'D:/Downloads/Dataset'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Create ImageDataGenerator for training, validation, and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    data_dir + '/val',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    data_dir + '/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained DenseNet121 model
base_model_densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model_densenet.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
predictions_densenet = Dense(6, activation='softmax')(x)

# Create the DenseNet121 model
model_densenet = Model(inputs=base_model_densenet.input, outputs=predictions_densenet)

# Freeze the layers in the base model
for layer in base_model_densenet.layers:
    layer.trainable = False

# Compile the model
model_densenet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Load pre-trained InceptionV3 model
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer
x = base_model_inception.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
predictions_inception = Dense(6, activation='softmax')(x)

# Create the InceptionV3 model
model_inception = Model(inputs=base_model_inception.input, outputs=predictions_inception)

# Freeze the layers in the base model
for layer in base_model_inception.layers:
    layer.trainable = False

# Compile the model
model_inception.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Combine the models into an ensemble model
combined_input = tf.keras.layers.concatenate([model_densenet.output, model_inception.output])
ensemble_output = Dense(6, activation='softmax')(combined_input)
ensemble_model = Model(inputs=[model_densenet.input, model_inception.input], outputs=ensemble_output)

# Compile the ensemble model
ensemble_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble model
# Train the ensemble model
# Train the ensemble model
ensemble_model.fit(
    {"input_1": train_generator, "input_2": train_generator},
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=({"input_1": val_generator, "input_2": val_generator}, None),
    validation_steps=len(val_generator)
)



# Evaluate the model on the test set
test_loss, test_accuracy = ensemble_model.evaluate([test_generator, test_generator], steps=len(test_generator))
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')


Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.
Found 4110 images belonging to 6 classes.


ValueError: Failed to find data adapter that can handle input: (<class 'dict'> containing {"<class 'str'>"} keys and {"<class 'keras.src.preprocessing.image.DirectoryIterator'>"} values), <class 'NoneType'>